# DEMO 2 
## Draw Random Body Poses from the Learned Pose Prior Distributions

Provided a body in the canonical space, we draw random samples from the pose prior, and then pose the body via the differentiable forward kinematics.

Note:
- the `LISST` package should be pre-installed.
- reading `demo1` first is highly recommended.

In [1]:
# load necessary modules
import os, sys, glob
import numpy as np
import torch
from lisst.models.baseops import RotConverter
from lisst.utils.config_creator import ConfigLoader
from lisst.models.body import LISSTCore
from lisst.models.body import LISSTPoser

DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## The LISST body and pose configurations

### Recap the body shape model
As introduced in *Demo1*, there are three body shape configures:
- `LISST_SHAPER_v0`: has the same skeleton topology with the CMU skeleton
- `LISST_SHAPER_v1`: additionally add the nose joint to better represent the head rotation. See documents.
- `LISST_SHAPER_v2`: additionally add the nose and the heels.

Here we use `LISST_SHAPER_v2` as an example.

### The body posers
- The body poser learns the distribution of joint rotations in terms of the 6D continuous representation [Zhou et al, CVPR'19].
- The body is just posed in the body coordinate system, without global transformation.
- All rotations are w.r.t. the pelvis, instead of their parents.

The poser architectures:
- `LISST_POSER_v0`: MLP-based

However, the pose priors are only learned with the CMU mocap skeleton. The nose and heel joints are not included. Instead, the nose joint has the same transformation with the head, and the heel joints have the same transformations with the feet, respectively, because they are welded to their parents.

In [2]:
shaper_config_name = 'LISST_SHAPER_v2'
poser_config_name = 'LISST_POSER_v0'


# create shaper and load checkpoint
shaper_config = ConfigLoader('/home/yzhang/workspaces/LISST-dev/lisst/cfg/{}.yml'.format(shaper_config_name))
shaper = LISSTCore(shaper_config.modelconfig)
shaper.eval()
shaper.to(DEVICE)
## load checkpoint
shaper_ckpt = '/home/yzhang/workspaces/LISST-dev/results/lisst/{}/checkpoints/epoch-000.ckp'.format(shaper_config_name)
shaper.load(shaper_ckpt)

# create poser and load checkpoint
poser_config = ConfigLoader('/home/yzhang/workspaces/LISST-dev/lisst/cfg/{}.yml'.format(poser_config_name))
poser = LISSTPoser(poser_config.modelconfig)
poser.eval()
poser.to(DEVICE)
## load checkpoint
poser_ckpt = '/home/yzhang/workspaces/LISST-dev/results/lisst/{}/checkpoints/epoch-500.ckp'.format(poser_config_name)
poser.load(poser_ckpt)



-- successfully loaded: /home/yzhang/workspaces/LISST-dev/results/lisst/LISST_SHAPER_v2/checkpoints/epoch-000.ckp
-- successfully loaded: /home/yzhang/workspaces/LISST-dev/results/lisst/LISST_POSER_v1/checkpoints/epoch-500.ckp


## sample random poses of a specific body

Rather than using an encapsulated function, here we show the details of implementations.


In [3]:
n_poses = 10
n_bodies = 2

# draw a random body from the shape PCA space
zs = torch.zeros(n_bodies,shaper.num_kpts).to(DEVICE)
zs[:,:15] = 50*torch.randn(n_bodies, 15) # the first 15 pcs are considered
bone_length = shaper.decode(zs).repeat(n_poses,1)

# generate body poses
nj_poser = 31 # the poser only learns poses of the 31 joints in the CMU mocap data.
zp = torch.randn(n_bodies*n_poses, nj_poser, poser.z_dim).to(DEVICE) #[b,J,d]
poses_rotcont = poser.decode(zp).reshape(n_bodies*n_poses, -1)
poses_rotcont_reshape = poses_rotcont.reshape(-1,nj_poser, 6)
## add additional body parts if additional bones are considered
poses_rotcont_reshape = poser.add_additional_bones(poses_rotcont_reshape, shaper.joint_names,
                                            new_joints=shaper.get_new_joints())
## change 6D representation to rotation matrix
poses_rotcont_reshape = poses_rotcont_reshape.reshape(n_bodies*n_poses*shaper.num_kpts, 6)
poses_rotmat = RotConverter.cont2rotmat(poses_rotcont_reshape).reshape(n_bodies*n_poses, shaper.num_kpts, 3,3)

# forward kinematics
x_root = torch.zeros(n_bodies*n_poses, 1,3).to(DEVICE)
J_locs_fk = shaper.forward_kinematics(x_root, bone_length,poses_rotmat)
J_locs_fk = J_locs_fk.contiguous().view(n_bodies, n_poses, -1, 3).detach().cpu().numpy()


In [8]:
# visualization
# J_locs_fk has [n_bodies, n_poses, n_joints, 3]
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(J_locs_fk[0,8])
pcd.paint_uniform_color([0, 0, 0])
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.25)
o3d.visualization.draw_geometries([pcd, coord])

